# Objetivo:
#### Entrenamiento, evaluación, validación y comparaciones de modelos ML y DL

# Importar librerias

In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import scipy.stats as stats
from sklearn.model_selection import train_test_split, GridSearchCV
import statsmodels.api as sm
import numpy as np
from sklearn.model_selection import cross_val_score
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit

/usr/local/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-07-07 18:34:48.865396: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import LabelEncoder

# 0. Get data

In [3]:
data = pd.read_csv('../data/final_data/AllData_Filtered.csv')
data.head()

,Median_rooms,Total_population,Renter_occupied,Total_Household,PerCapita_income,Nonfamily_households,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,Median_Price,State,Year
0,0.368421,0.052806,0.027643,0.057499,0.422496,0.073425,0.644039,0.052458,0.046914,0.371501,Florida,2011
1,0.342105,0.138650,0.089285,0.160742,0.373741,0.222352,0.552178,0.138591,0.119097,0.323289,Florida,2011
2,0.342105,0.051212,0.021888,0.060228,0.192084,0.069472,0.403580,0.051068,0.052038,0.170490,Florida,2011
3,0.500000,0.054498,0.029178,0.064072,0.255898,0.047479,0.297197,0.054006,0.046796,0.245529,Florida,2011
4,0.210526,0.641212,0.469767,0.648822,0.295477,0.844463,0.515029,0.644490,0.648317,0.346423,Florida,2011


In [4]:
columns_to_dum = ['State']#, 'Year']
for column in columns_to_dum:
    dummies = pd.get_dummies(data[column], prefix=column, drop_first=True).astype(int)
    data = pd.concat([data, dummies], axis=1)
data.head()

,Median_rooms,Total_population,Renter_occupied,Total_Household,PerCapita_income,Nonfamily_households,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,Median_Price,State,Year,State_New Jersey,State_New York,State_North Carolina,State_South Carolina,State_Virginia
0,0.368421,0.052806,0.027643,0.057499,0.422496,0.073425,0.644039,0.052458,0.046914,0.371501,Florida,2011,0,0,0,0,0
1,0.342105,0.138650,0.089285,0.160742,0.373741,0.222352,0.552178,0.138591,0.119097,0.323289,Florida,2011,0,0,0,0,0
2,0.342105,0.051212,0.021888,0.060228,0.192084,0.069472,0.403580,0.051068,0.052038,0.170490,Florida,2011,0,0,0,0,0
3,0.500000,0.054498,0.029178,0.064072,0.255898,0.047479,0.297197,0.054006,0.046796,0.245529,Florida,2011,0,0,0,0,0
4,0.210526,0.641212,0.469767,0.648822,0.295477,0.844463,0.515029,0.644490,0.648317,0.346423,Florida,2011,0,0,0,0,0


In [5]:
columns_to_encode = ['Year']
for column in columns_to_encode:
    le = LabelEncoder()
    data[column + '_encoded'] = le.fit_transform(data[column])
data.head()

,Median_rooms,Total_population,Renter_occupied,Total_Household,PerCapita_income,Nonfamily_households,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,Median_Price,State,Year,State_New Jersey,State_New York,State_North Carolina,State_South Carolina,State_Virginia,Year_encoded
0,0.368421,0.052806,0.027643,0.057499,0.422496,0.073425,0.644039,0.052458,0.046914,0.371501,Florida,2011,0,0,0,0,0,0
1,0.342105,0.138650,0.089285,0.160742,0.373741,0.222352,0.552178,0.138591,0.119097,0.323289,Florida,2011,0,0,0,0,0,0
2,0.342105,0.051212,0.021888,0.060228,0.192084,0.069472,0.403580,0.051068,0.052038,0.170490,Florida,2011,0,0,0,0,0,0
3,0.500000,0.054498,0.029178,0.064072,0.255898,0.047479,0.297197,0.054006,0.046796,0.245529,Florida,2011,0,0,0,0,0,0
4,0.210526,0.641212,0.469767,0.648822,0.295477,0.844463,0.515029,0.644490,0.648317,0.346423,Florida,2011,0,0,0,0,0,0


In [6]:
data.shape

(4878, 18)

In [7]:
data.columns

Index(['Median_rooms', 'Total_population', 'Renter_occupied',
       'Total_Household', 'PerCapita_income', 'Nonfamily_households',
       'Gini_Index', 'Poverty_Status', 'Unemployment_ 16YearsAndOver',
       'Median_Price', 'State', 'Year', 'State_New Jersey', 'State_New York',
       'State_North Carolina', 'State_South Carolina', 'State_Virginia',
       'Year_encoded'],
      dtype='object')

In [8]:
data.dtypes

Median_rooms                    float64
Total_population                float64
Renter_occupied                 float64
Total_Household                 float64
PerCapita_income                float64
Nonfamily_households            float64
Gini_Index                      float64
Poverty_Status                  float64
Unemployment_ 16YearsAndOver    float64
Median_Price                    float64
State                            object
Year                              int64
State_New Jersey                  int64
State_New York                    int64
State_North Carolina              int64
State_South Carolina              int64
State_Virginia                    int64
Year_encoded                      int64
dtype: object

In [9]:
data.Year_encoded.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

# 1. Split data

In [10]:
data.groupby('Year').count()

,Median_rooms,Total_population,Renter_occupied,Total_Household,PerCapita_income,Nonfamily_households,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,Median_Price,State,State_New Jersey,State_New York,State_North Carolina,State_South Carolina,State_Virginia,Year_encoded
Year,,,,,,,,,,,,,,,,,
2011,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410
2012,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412
2013,413,413,413,413,413,413,413,413,413,413,413,413,413,413,413,413,413
2014,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412
2015,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411
2016,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411
2017,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411
2018,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410
2019,406,406,406,406,406,406,406,406,406,406,406,406,406,406,406,406,406


In [11]:
data_val = data[data['Year'] == 2022]
data_model = data[data['Year'] != 2022]
data_model.shape

(4502, 18)

In [12]:
X_val = data_val.drop(['State','Median_Price','Year'], axis=1)
y_val = data_val['Median_Price']
X_val.shape

(376, 15)

In [13]:
data_model['Year'] = pd.to_datetime(data_model['Year'], format='%Y')
data_model['Year'] = data_model['Year'].dt.year
data_model = data_model.sort_values(by='Year').reset_index(drop=True)
data_model.head()

/var/folders/rv/dlx920mn2hj15wkt3g1nkdkn3dpgn4/T/ipykernel_31780/746859498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_model['Year'] = pd.to_datetime(data_model['Year'], format='%Y')
/var/folders/rv/dlx920mn2hj15wkt3g1nkdkn3dpgn4/T/ipykernel_31780/746859498.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_model['Year'] = data_model['Year'].dt.year


,Median_rooms,Total_population,Renter_occupied,Total_Household,PerCapita_income,Nonfamily_households,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,Median_Price,State,Year,State_New Jersey,State_New York,State_North Carolina,State_South Carolina,State_Virginia,Year_encoded
0,0.368421,0.052806,0.027643,0.057499,0.422496,0.073425,0.644039,0.052458,0.046914,0.371501,Florida,2011,0,0,0,0,0,0
1,0.289474,0.017771,0.020488,0.016264,0.162163,0.031894,0.654846,0.015936,0.015780,0.363919,North Carolina,2011,0,0,1,0,0,0
2,0.394737,0.019454,0.017607,0.020929,0.102570,0.024582,0.500169,0.019119,0.024223,0.076205,North Carolina,2011,0,0,1,0,0,0
3,0.368421,0.054265,0.041521,0.062066,0.201706,0.065542,0.405944,0.053620,0.043818,0.172823,North Carolina,2011,0,0,1,0,0,0
4,0.473684,0.000802,0.000373,0.000841,0.082038,0.000687,0.250929,0.000583,0.002782,0.121501,North Carolina,2011,0,0,1,0,0,0


In [14]:
split_index = int(len(data_model) * 0.8)
split_index

3601

In [15]:
data_model.Year = data_model.Year.astype(int)

In [16]:
data_train = data_model[:split_index]
data_test = data_model[split_index:]

In [17]:
data_train.Year.unique(), data_test.Year.unique()

(array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]),
 array([2019, 2020, 2021]))

In [18]:
data_train.describe()

,Median_rooms,Total_population,Renter_occupied,Total_Household,PerCapita_income,Nonfamily_households,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,Median_Price,Year,State_New Jersey,State_New York,State_North Carolina,State_South Carolina,State_Virginia,Year_encoded
count,3601.000000,3.601000e+03,3601.000000,3601.000000,3601.000000,3601.000000,3601.000000,3601.000000,3601.000000,3601.000000,3601.000000,3601.000000,3601.000000,3601.000000,3601.000000,3601.000000,3601.000000
mean,0.453083,5.165161e-02,0.040135,0.053802,0.237810,0.061195,0.425984,0.051006,0.038890,0.222675,2014.886976,0.052485,0.135796,0.227159,0.101916,0.318245,3.886976
std,0.122575,9.690038e-02,0.088308,0.096570,0.117018,0.113768,0.115191,0.096607,0.079382,0.145271,2.528010,0.223035,0.342619,0.419054,0.302580,0.465860,2.528010
min,0.000000,7.371060e-07,0.000029,0.000110,0.000000,0.000000,0.000000,0.000000,0.000000,0.009720,2011.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.368421,7.460250e-03,0.004447,0.007836,0.155044,0.008355,0.349544,0.007223,0.005177,0.113725,2013.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,0.447368,1.802187e-02,0.012150,0.019138,0.219144,0.021922,0.423168,0.017417,0.012677,0.190124,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
75%,0.526316,5.025552e-02,0.034884,0.054257,0.298499,0.054907,0.501182,0.049713,0.036979,0.291213,2017.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000
max,1.000000,1.000000e+00,0.958035,0.942078,0.821097,0.946202,0.947991,1.000000,1.000000,0.830288,2019.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000


In [19]:
data_test.describe()

,Median_rooms,Total_population,Renter_occupied,Total_Household,PerCapita_income,Nonfamily_households,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,Median_Price,Year,State_New Jersey,State_New York,State_North Carolina,State_South Carolina,State_Virginia,Year_encoded
count,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000
mean,0.462118,0.048564,0.037686,0.050878,0.321355,0.060182,0.450552,0.047959,0.023861,0.251528,2020.338513,0.046615,0.117647,0.275250,0.135405,0.281909,9.338513
std,0.120485,0.091523,0.082400,0.092782,0.133764,0.111828,0.117437,0.091361,0.046340,0.155020,0.659759,0.210930,0.322369,0.446888,0.342346,0.450179,0.659759
min,0.026316,0.000000,0.000000,0.000000,0.059610,0.000383,0.054711,0.000002,0.000151,0.000000,2019.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000
25%,0.394737,0.007411,0.004591,0.007665,0.223454,0.009275,0.380952,0.007177,0.003358,0.138025,2020.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000
50%,0.447368,0.016889,0.011803,0.018003,0.299751,0.022346,0.450523,0.016527,0.008541,0.220062,2020.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000
75%,0.526316,0.048057,0.033293,0.051282,0.389784,0.057950,0.521108,0.047713,0.022364,0.336120,2021.000000,0.000000,0.000000,1.000000,0.000000,1.000000,10.000000
max,1.000000,0.996319,1.000000,1.000000,0.889167,1.000000,1.000000,0.996934,0.468987,0.860226,2021.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.000000


In [20]:
data_train.groupby('Year').count()

,Median_rooms,Total_population,Renter_occupied,Total_Household,PerCapita_income,Nonfamily_households,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,Median_Price,State,State_New Jersey,State_New York,State_North Carolina,State_South Carolina,State_Virginia,Year_encoded
Year,,,,,,,,,,,,,,,,,
2011,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410
2012,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412
2013,413,413,413,413,413,413,413,413,413,413,413,413,413,413,413,413,413
2014,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412,412
2015,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411
2016,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411
2017,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411,411
2018,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410
2019,311,311,311,311,311,311,311,311,311,311,311,311,311,311,311,311,311


In [21]:
data_test.groupby('Year').count()

,Median_rooms,Total_population,Renter_occupied,Total_Household,PerCapita_income,Nonfamily_households,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,Median_Price,State,State_New Jersey,State_New York,State_North Carolina,State_South Carolina,State_Virginia,Year_encoded
Year,,,,,,,,,,,,,,,,,
2019,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95,95
2020,406,406,406,406,406,406,406,406,406,406,406,406,406,406,406,406,406
2021,400,400,400,400,400,400,400,400,400,400,400,400,400,400,400,400,400


In [22]:
X_train = data_train.drop(['State','Median_Price','Year'], axis=1)
y_train = data_train['Median_Price']
X_train.shape

(3601, 15)

In [23]:
X_test = data_test.drop(['State','Median_Price','Year'], axis=1)
y_test = data_test['Median_Price']
X_test.shape

(901, 15)

In [24]:
# Divide el dataset en conjuntos de entrenamiento y prueba
#X = data_model.drop(['State','Median_Price','Year'], axis=1)
#y = data_model['Median_Price']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [25]:
X_train.shape, X_test.shape

((3601, 15), (901, 15))

# 2. Regresion multiple lineal como modelo base de comparacion

In [26]:
# Eliminar la columna 'Total_Household' de los conjuntos de datos
X_train_rg = X_train.drop(['Total_Household','Poverty_Status'], axis=1)
X_test_rg = X_test.drop(['Total_Household', 'Poverty_Status'], axis=1)
X_val_rg = X_val.drop(['Total_Household', 'Poverty_Status'], axis=1)

# Convertir y_train a arrays de numpy
y_train_np = y_train.values

# Inicializar el modelo de regresión lineal
model = LinearRegression()

# Configuración de la validación cruzada con TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Listas para almacenar los coeficientes y errores de validación cruzada
coefficients = []
cross_val_errors = []

# Realiza la validación cruzada en orden cronológico
for train_index, val_index in tscv.split(X_train_rg):
    X_train_fold, X_val_fold = X_train_rg.iloc[train_index], X_train_rg.iloc[val_index]
    y_train_fold, y_val_fold = y_train_np[train_index], y_train_np[val_index]
    
    model.fit(X_train_fold, y_train_fold)
    coefficients.append(model.coef_)
    
    y_pred_val_fold = model.predict(X_val_fold)
    mse_val_fold = mean_squared_error(y_val_fold, y_pred_val_fold)
    cross_val_errors.append(mse_val_fold)

# Calcula la media de los errores de validación cruzada
mse_cross_validation = np.mean(cross_val_errors)
rmse_cross_validation = np.sqrt(mse_cross_validation)

# Entrena el modelo en el conjunto de entrenamiento completo
model.fit(X_train_rg, y_train)

# Predicción y evaluación en el conjunto de train
y_pred_train = model.predict(X_train_rg)
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

# Predicción y evaluación en el conjunto de prueba
y_pred_test = model.predict(X_test_rg)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

# Predicción y evaluación en el conjunto de validación final
y_pred_val = model.predict(X_val_rg)
mse_val = mean_squared_error(y_val, y_pred_val)
r2_val = r2_score(y_val, y_pred_val)

# Utiliza statsmodels para obtener los valores p
X_train_sm = sm.add_constant(X_train_rg)  # Agrega una constante para la intersección
ols_model = sm.OLS(y_train, X_train_sm).fit()

coefs = ols_model.params
pvalues = ols_model.pvalues

# Crea una tabla con los coeficientes y p-values
coef_table = pd.DataFrame({
    'Coefficient': coefs,
    'P-value': pvalues
})

# Almacena la información del modelo
model_info = {
    'version': 'Regresión Lineal con Validación Cruzada Cronológica',
    'model_name': model.__class__.__name__,
    'rmse_train': np.sqrt(mse_train),
    'r2_train': r2_train,
    'rmse_test': np.sqrt(mse_test),
    'r2_test': r2_test,
    'rmse_val': np.sqrt(mse_val),
    'r2_val': r2_val
}

In [27]:
df_models_info = pd.DataFrame([model_info])
df_models_info

,version,model_name,rmse_train,r2_train,rmse_test,r2_test,rmse_val,r2_val
0,Regresión Lineal con Validación Cruzada Cronol...,LinearRegression,0.059002,0.834998,0.06361,0.831437,0.071444,0.829417


In [28]:
df_models_info.to_excel('../data/final_data/v2_regresion_lineal_info.xlsx', index=False)

In [29]:
coef_table

,Coefficient,P-value
const,0.048759,2.619782e-10
Median_rooms,-0.101241,3.405614e-13
Total_population,0.614895,1.861484e-22
Renter_occupied,0.858786,5.943582e-86
PerCapita_income,1.100580,0.000000e+00
Nonfamily_households,-1.040377,2.411983e-120
Gini_Index,-0.039417,2.138485e-04
Unemployment_ 16YearsAndOver,-0.077962,1.028391e-01
State_New Jersey,0.055964,2.401478e-22
State_New York,-0.050412,1.477026e-35


In [30]:
coef_table.to_excel('../data/final_data/v2_regresion_lineal_coef.xlsx', index=False)

# 3. Modelos con parametros base, sin optimizacion

In [31]:
data.Total_Household.mean()

0.052746701832372234

In [32]:
# Definir el wrapper CustomKerasRegressor
class CustomKerasRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, build_fn=None, **kwargs):
        self.build_fn = build_fn
        self.kwargs = kwargs
        self.model = None

    def fit(self, X, y, **fit_params):
        self.model = self.build_fn(**self.kwargs)
        early_stopping = EarlyStopping(monitor="loss", patience=self.kwargs.get('patience', 5))
        self.model.fit(X, y,
                       batch_size=self.kwargs.get('batch_size', 32),
                       epochs=self.kwargs.get('epochs', 50),
                       callbacks=[early_stopping],
                       verbose=self.kwargs.get('verbose', 0),
                       **fit_params)
        return self

    def predict(self, X):
        return self.model.predict(X)

    def get_params(self, deep=True):
        return {"build_fn": self.build_fn, **self.kwargs}

    def set_params(self, **params):
        if "build_fn" in params:
            self.build_fn = params.pop("build_fn")
        self.kwargs.update(params)
        return self

In [33]:
def ANN(optimizer='adam', neurons=32, activation='relu', loss='mse', **kwargs):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X_train.shape[1],), activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss=loss)
    return model

In [34]:
models = [
    RandomForestRegressor(random_state=0),
    Lasso(),
    KNeighborsRegressor(),
    GradientBoostingRegressor(random_state=0),
    AdaBoostRegressor(random_state=0),
    XGBRegressor(random_state=0),
    CustomKerasRegressor(build_fn=ANN),
]

In [35]:

scaler = StandardScaler()

# Ajustar y transformar los conjuntos de datos
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

models_info = []

# Configuración de la validación cruzada con TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Proceso de entrenamiento y evaluación con validación cruzada
for model in models:
    cross_val_errors = []

    # Realiza la validación cruzada en orden cronológico
    for train_index, val_index in tscv.split(X_train_scaled):
        X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model.fit(X_train_fold, y_train_fold)
        
        y_pred_val_fold = model.predict(X_val_fold)
        mse_val_fold = mean_squared_error(y_val_fold, y_pred_val_fold)
        cross_val_errors.append(mse_val_fold)
    
    # Calcula la media de los errores de validación cruzada
    mse_cross_validation = np.mean(cross_val_errors)
    rmse_cross_validation = np.sqrt(mse_cross_validation)

    # Entrena el modelo en el conjunto de entrenamiento completo (sin escalar)
    model.fit(X_train_scaled, y_train)

    # Predicción y evaluación en el conjunto de entrenamiento
    y_pred_train = model.predict(X_train_scaled)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)

    # Predicción y evaluación en el conjunto de prueba
    y_pred_test = model.predict(X_test_scaled)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)

    # Predicción y evaluación en el conjunto de validación final
    y_pred_val = model.predict(X_val_scaled)
    mse_val = mean_squared_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    # Almacena la información del modelo
    model_info = {
        'version': 'Modelos con parámetros por defecto',
        'model_name': model.__class__.__name__,
        'rmse_cross_validation': rmse_cross_validation,
        'rmse_train': np.sqrt(mse_train),
        'r2_train': r2_train,
        'rmse_test': np.sqrt(mse_test),
        'r2_test': r2_test,
        'rmse_val': np.sqrt(mse_val),
        'r2_val': r2_val
    }

    models_info.append(model_info)

12/12 [==============================] - 0s 988us/step


In [36]:
df_models_info = pd.DataFrame(models_info)
df_models_info

,version,model_name,rmse_cross_validation,rmse_train,r2_train,rmse_test,r2_test,rmse_val,r2_val
0,Modelos con parámetros por defecto,RandomForestRegressor,0.050187,0.013727,0.991068,0.067873,0.808090,0.098960,0.672718
1,Modelos con parámetros por defecto,Lasso,0.143120,0.145251,0.000000,0.157598,-0.034683,0.191246,-0.222337
2,Modelos con parámetros por defecto,KNeighborsRegressor,0.051204,0.034115,0.944837,0.060221,0.848920,0.086029,0.752661
3,Modelos con parámetros por defecto,GradientBoostingRegressor,0.049265,0.041429,0.918648,0.063323,0.832955,0.082980,0.769880
4,Modelos con parámetros por defecto,AdaBoostRegressor,0.071892,0.068327,0.778720,0.092021,0.647238,0.107970,0.610411
5,Modelos con parámetros por defecto,XGBRegressor,0.043181,0.007339,0.997447,0.054932,0.874293,0.086203,0.751656
6,Modelos con parámetros por defecto,CustomKerasRegressor,0.055900,0.038783,0.928707,0.055839,0.870109,0.072882,0.822481


In [37]:
df_models_info.to_excel('../data/final_data/v2_modelos_default.xlsx', index=False)

# 4. Modelos con RandomizedSearchCV para optimizacion

In [38]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
# Parámetros de búsqueda para RandomForestRegressor
rf_params = {
    'n_estimators': sp_randint(10, 100),
    'max_features': sp_randint(1, 13),
    'max_depth': sp_randint(5, 50),
    'min_samples_split': sp_randint(2, 11),
    'min_samples_leaf': sp_randint(1, 11),
    'criterion': ['mse', 'mae']
}

# Parámetros de búsqueda para Lasso
lasso_params = {
    'alpha': sp_randint(1, 100)
}

# Parámetros de búsqueda para KNeighborsRegressor
knn_params = {
    'n_neighbors': sp_randint(1, 30),
    'weights': ['uniform', 'distance'],
    'p': sp_randint(1, 2)
}

# Parámetros de búsqueda para GradientBoostingRegressor
gb_params = {
    'n_estimators': sp_randint(10, 200),
    'max_depth': sp_randint(1, 10),
    'learning_rate': [0.01, 0.1],
    'subsample': [0.5, 1.0]
}

# Parámetros de búsqueda para AdaBoostRegressor
ada_params = {
    'n_estimators': sp_randint(10, 200),
    'learning_rate': [0.01, 0.1, 1.0]
}

# Parámetros de búsqueda para XGBRegressor
xgb_params = {
    'n_estimators': sp_randint(10, 200),
    'max_depth': sp_randint(1, 10),
    'learning_rate': [0.01, 0.1],
    'subsample': [0.5, 1.0],
    'colsample_bytree': [0.5, 1.0]
}

# Parámetros de búsqueda para CustomKerasRegressor
cus_keras_ann_params = {
    'optimizer': ['adam', 'rmsprop'],
    'activation': ['relu', 'tanh'],
    'loss': ['mse', 'mae'],
    'batch_size': sp_randint(16, 32),
    'neurons': sp_randint(16, 32),
    'epochs': sp_randint(20, 50),
    'patience': sp_randint(2, 5)
}

In [39]:
models_params = [
    (RandomForestRegressor(random_state=0), rf_params),
    (Lasso(), lasso_params),
    (KNeighborsRegressor(), knn_params),
    (GradientBoostingRegressor(random_state=0), gb_params),
    (AdaBoostRegressor(random_state=0), ada_params),
    (XGBRegressor(random_state=0), xgb_params),
    (CustomKerasRegressor(build_fn=ANN), cus_keras_ann_params)
]

# Inicializar el escalador
scaler = StandardScaler()

# Ajustar y transformar los conjuntos de datos
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

# Diccionario para guardar la información de los modelos
models_info = []

# Configuración de la validación cruzada con TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Proceso de búsqueda de hiperparámetros y evaluación
for model, params in models_params:
    search = RandomizedSearchCV(model, params, n_iter=50, cv=tscv, scoring='neg_mean_squared_error', random_state=0, n_jobs=-1)
    search.fit(X_train_scaled, y_train)
    
    best_model = search.best_estimator_

    # Predicción y evaluación en el conjunto de entrenamiento
    y_pred_train = best_model.predict(X_train_scaled)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    
    y_pred_test = best_model.predict(X_test_scaled)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    y_pred_val = best_model.predict(X_val_scaled)
    mse_val = mean_squared_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    
    model_info = {
        "version": "Modelos con hiperparámetros optimizados usando RandomizedSearchCV y TimeSeriesSplit",
        'model_name': best_model.__class__.__name__,
        'best_params': search.best_params_,
        'rmse_train': np.sqrt(mse_train),
        'r2_train': r2_train,
        'rmse_test': np.sqrt(mse_test),
        'r2_test': r2_test,
        'rmse_val': np.sqrt(mse_val),
        'r2_val': r2_val
    }
    
    models_info.append(model_info)

/usr/local/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr

19/19 [==============================] - 0s 2ms/step


/usr/local/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


19/19 [==============================] - 0s 2ms/step


/usr/local/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


...

19/19 [==============================] - 0s 1ms/step


/usr/local/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-07-07 18:38:21.006310: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn

19/19 [==============================] - 0s 2ms/step


2024-07-07 18:38:31.501364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-07 18:38:32.622016: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


12/12 [==============================] - 0s 1ms/step


In [40]:
df_models_info = pd.DataFrame(models_info)
df_models_info

,version,model_name,best_params,rmse_train,r2_train,rmse_test,r2_test,rmse_val,r2_val
0,Modelos con hiperparámetros optimizados usando...,RandomForestRegressor,"{'criterion': 'mae', 'max_depth': 45, 'max_fea...",0.013014,0.991973,0.056850,0.865360,0.083455,0.767241
1,Modelos con hiperparámetros optimizados usando...,Lasso,{'alpha': 45},0.145251,0.000000,0.157598,-0.034683,0.191246,-0.222337
2,Modelos con hiperparámetros optimizados usando...,KNeighborsRegressor,"{'n_neighbors': 2, 'p': 1, 'weights': 'distance'}",0.000000,1.000000,0.051548,0.889303,0.096474,0.688952
3,Modelos con hiperparámetros optimizados usando...,GradientBoostingRegressor,"{'learning_rate': 0.1, 'max_depth': 8, 'n_esti...",0.004455,0.999059,0.056630,0.866403,0.087954,0.741467
4,Modelos con hiperparámetros optimizados usando...,AdaBoostRegressor,"{'learning_rate': 1.0, 'n_estimators': 57}",0.067710,0.782699,0.090973,0.655227,0.107111,0.616579
5,Modelos con hiperparámetros optimizados usando...,XGBRegressor,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",0.013093,0.991874,0.053422,0.881111,0.079175,0.790500
6,Modelos con hiperparámetros optimizados usando...,CustomKerasRegressor,"{'activation': 'relu', 'batch_size': 17, 'epoc...",0.047244,0.894209,0.059382,0.853101,0.075551,0.809241


In [41]:
df_models_info.to_excel('../data/final_data/v2_models_hiperparametros_randomsearch.xlsx', index=False)

# 5. Modelos con GridSearchCV para optimziacion

In [42]:
rf_params = {
    'n_estimators': [10, 20, 30, 40, 50],
    'max_depth': [15, 20, 30, 50],
    'max_features': [3,5,7,10],
    'min_samples_split': [3,5,7,10],
    'min_samples_leaf': [1,3,5],
    'criterion': ['mse', 'mae']
}
lasso_params = {
    'alpha': [0.05, 0.1, 0.5, 1.0, 5.0, 10.0]
}

knn_params = {
    'n_neighbors': [3, 5, 7, 10,15,20],
    'weights': ['uniform', 'distance'],
    'p': [1,2]
}

gb_params = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.5, 1.0]
}

ada_params = {
    'n_estimators': [50, 100,150, 200],
    'learning_rate': [0.01, 0.1, 0.2]
}

xgb_params = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.5, 1.0],
    'colsample_bytree': [0.5, 1.0]
}

cus_keras_ann_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu','tanh'],
    'loss': ['mse','mae'],
    'batch_size': [16,23,32],
    'neurons':[16,23,32],
    'epochs':[20,35,50],
    'patience':[2,3,5]
 
}

In [43]:
# Lista de modelos y sus hiperparámetros
models_params = [
    (RandomForestRegressor(random_state=0), rf_params),
    (Lasso(), lasso_params),
    (KNeighborsRegressor(), knn_params),
    (GradientBoostingRegressor(random_state=0), gb_params),
    (AdaBoostRegressor(random_state=0), ada_params),
    (XGBRegressor(random_state=0), xgb_params),
    (CustomKerasRegressor(build_fn=ANN), cus_keras_ann_params)
]

In [44]:
# Inicializar el escalador
scaler = StandardScaler()

# Ajustar y transformar los conjuntos de datos
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

models_info = []
#models_params = [(CustomKerasRegressor(build_fn=ANN), cus_keras_ann_params)]

# Configuración de la validación cruzada con TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Proceso de búsqueda de hiperparámetros y evaluación
for model, params in models_params:
    grid = GridSearchCV(model, params, cv=tscv, scoring='neg_mean_squared_error')
    grid.fit(X_train_scaled, y_train)
    
    best_model = grid.best_estimator_

    # Predicción y evaluación en el conjunto de entrenamiento
    y_pred_train = best_model.predict(X_train_scaled)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    
    y_pred_test = best_model.predict(X_test_scaled)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    y_pred_val = best_model.predict(X_val_scaled)
    mse_val = mean_squared_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    
    model_info = {
        "version": "Modelos con hiperparámetros optimizados usando GridSearchCV y TimeSeriesSplit",
        'model_name': best_model.__class__.__name__,
        'best_params': grid.best_params_,
        'rmse_train': np.sqrt(mse_train),
        'r2_train': r2_train,
        'rmse_test': np.sqrt(mse_test),
        'r2_test': r2_test,
        'rmse_val': np.sqrt(mse_val),
        'r2_val': r2_val
    }
    
    models_info.append(model_info)

/usr/local/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was d

19/19 [==============================] - 0s 1ms/step


KeyboardInterrupt: 

In [52]:
grid.fit

<bound method BaseSearchCV.fit of GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=CustomKerasRegressor(build_fn=<function ANN at 0x14c28e040>),
             param_grid={'activation': ['relu', 'tanh'],
                         'batch_size': [16, 23, 32], 'epochs': [20, 35, 50],
                         'loss': ['mse', 'mae'], 'neurons': [16, 23, 32],
                         'optimizer': ['adam', 'rmsprop'],
                         'patience': [2, 3, 5]},
             scoring='neg_mean_squared_error')>

In [45]:
df_models_info = pd.DataFrame(models_info)
df_models_info

,version,model_name,best_params,rmse_train,r2_train,rmse_test,r2_test,rmse_val,r2_val
0,Modelos con hiperparámetros optimizados usando...,RandomForestRegressor,"{'criterion': 'mae', 'max_depth': 20, 'max_fea...",0.012366,0.992753,0.053543,0.880569,0.081245,0.779406
1,Modelos con hiperparámetros optimizados usando...,Lasso,{'alpha': 0.05},0.093470,0.585896,0.094331,0.629306,0.104877,0.632408
2,Modelos con hiperparámetros optimizados usando...,KNeighborsRegressor,"{'n_neighbors': 3, 'p': 1, 'weights': 'distance'}",0.000000,1.000000,0.052338,0.885887,0.093389,0.708531
3,Modelos con hiperparámetros optimizados usando...,GradientBoostingRegressor,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",0.015397,0.988763,0.051982,0.887432,0.078198,0.795638
4,Modelos con hiperparámetros optimizados usando...,AdaBoostRegressor,"{'learning_rate': 0.2, 'n_estimators': 200}",0.068513,0.777514,0.092916,0.640341,0.110196,0.594180
5,Modelos con hiperparámetros optimizados usando...,XGBRegressor,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",0.008430,0.996632,0.050684,0.892986,0.074440,0.814809


In [46]:
df_models_info.to_excel('../data/final_data/v2modelos_hiperparametros_gridsearch.xlsx', index=False)

# 5. Best model

In [47]:
import pandas as pd
import plotly.express as px

In [53]:
# Eliminar la columna 'Total_Household' de los conjuntos de datos

X_train_rg = X_train.drop(['Total_Household','Poverty_Status'], axis=1)
X_test_rg = X_test.drop(['Total_Household', 'Poverty_Status'], axis=1)
X_val_rg = X_val.drop(['Total_Household', 'Poverty_Status'], axis=1)

# Utiliza statsmodels para obtener los valores p
X_train_sm = sm.add_constant(X_train_rg)  # Agrega una constante para la intersección
ols_model = sm.OLS(y_train, X_train_sm).fit()


# Obtenemos los coeficientes y valores p
coefs = ols_model.params
pvalues = ols_model.pvalues

# Creamos un DataFrame para mostrar los coeficientes y valores p de cada variable
variable_coefficients = pd.DataFrame({
    'Variable': X_train_rg.columns,
    'Coefficients': coefs[1:],  # Excluimos la constante
    'P-values': pvalues[1:]  # Excluimos la constante
})

# Ordenamos las variables por la magnitud del coeficiente
variable_coefficients = variable_coefficients.sort_values(by='Coefficients', ascending=False).reset_index(drop=True)
variable_coefficients



,Variable,Coefficients,P-values
0,PerCapita_income,1.100580,0.000000e+00
1,Renter_occupied,0.858786,5.943582e-86
2,Total_population,0.614895,1.861484e-22
3,State_New Jersey,0.055964,2.401478e-22
4,State_Virginia,0.041964,4.728113e-30
5,State_North Carolina,0.007486,2.994263e-02
6,Year_encoded,-0.008918,5.244237e-90
7,State_South Carolina,-0.011408,8.529748e-03
8,Gini_Index,-0.039417,2.138485e-04
9,State_New York,-0.050412,1.477026e-35


In [54]:
variable_coefficients.to_excel('../data/final_data/v2FINAL_coefficients.xlsx', index=False)

In [57]:
significant_variables = variable_coefficients[variable_coefficients['P-values'] <= 0.05]

# Ordenamos las variables por la magnitud del coeficiente
significant_variables = significant_variables.sort_values(by='Coefficients', ascending=False)

# Graficar los coeficientes de las variables significativas usando Plotly Express
fig = px.bar(significant_variables, x='Variable', y='Coefficients', title='Coeficientes de Variables Significativas en la Regresión Lineal')

# Mostrar el gráfico
fig.show()

In [58]:
X_train_rg.shape, X_test_rg.shape, X_val_rg.shape

((3601, 13), (901, 13), (376, 13))

In [59]:
variable_coefficients[variable_coefficients['P-values'] > 0.05]

,Variable,Coefficients,P-values
10,Unemployment_ 16YearsAndOver,-0.077962,0.102839


In [60]:
X_train_rg = X_train_rg.drop(['Unemployment_ 16YearsAndOver'], axis=1)
X_test_rg = X_test_rg.drop(['Unemployment_ 16YearsAndOver'], axis=1)
X_val_rg = X_val_rg.drop(['Unemployment_ 16YearsAndOver'], axis=1)


In [61]:
X_train_rg.shape, X_test_rg.shape, X_val_rg.shape

((3601, 12), (901, 12), (376, 12))

In [69]:
# Entrenar un nuevo modelo de regresión lineal con las variables significativas
X_train_sm = sm.add_constant(X_train_rg)  # Agrega una constante para la intersección

# Asegúrate de agregar la constante también a los datos de prueba y validación
X_test_sm = sm.add_constant(X_test_rg)
X_val_sm = X_val_rg.copy()
X_val_sm['const'] = 1.0

ols_model_sig = sm.OLS(y_train, X_train_sm).fit()

X_train_sm.shape, X_test_sm.shape, X_val_sm.shape

((3601, 13), (901, 13), (376, 13))

In [70]:
# Predicciones y residuales con el nuevo modelo
y_pred_train = ols_model_sig.predict(X_train_sm)
y_pred_test = ols_model_sig.predict(X_test_sm)
y_pred_val = ols_model_sig.predict(X_val_sm)

In [71]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [72]:

# Crear el gráfico de predicciones vs valores reales
fig = make_subplots(rows=1, cols=3, subplot_titles=('Entrenamiento', 'Prueba', 'Validación'))

fig.add_trace(go.Scatter(x=y_train, y=y_pred_train, mode='markers', name='Entrenamiento', marker=dict(color='blue')),
              row=1, col=1)
fig.add_trace(go.Scatter(x=y_test, y=y_pred_test, mode='markers', name='Prueba', marker=dict(color='green')),
              row=1, col=2)
fig.add_trace(go.Scatter(x=y_val, y=y_pred_val, mode='markers', name='Validación', marker=dict(color='red')),
              row=1, col=3)

fig.update_layout(title='Predicciones vs Valores Reales', xaxis_title='Valores Reales', yaxis_title='Predicciones')
fig.update_xaxes(title_text='Valores Reales', row=1, col=1)
fig.update_yaxes(title_text='Predicciones', row=1, col=1)
fig.update_xaxes(title_text='Valores Reales', row=1, col=2)
fig.update_xaxes(title_text='Valores Reales', row=1, col=3)

fig.show()

In [73]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Entrenamiento', 'Prueba', 'Validación'))

residuals_train = y_train - y_pred_train
residuals_test = y_test - y_pred_test
residuals_val = y_val - y_pred_val

fig.add_trace(go.Scatter(x=y_pred_train, y=residuals_train, mode='markers', name='Entrenamiento', marker=dict(color='blue')),
                  row=1, col=1)
fig.add_trace(go.Scatter(x=y_pred_test, y=residuals_test, mode='markers', name='Prueba', marker=dict(color='green')),
                  row=1, col=2)
fig.add_trace(go.Scatter(x=y_pred_val, y=residuals_val, mode='markers', name='Validación', marker=dict(color='red')),
                  row=1, col=3)

fig.update_layout(title='Residuales', xaxis_title='Predicciones', yaxis_title='Residuales')
fig.update_xaxes(title_text='Predicciones', row=1, col=1)
fig.update_yaxes(title_text='Residuales', row=1, col=1)
fig.update_xaxes(title_text='Predicciones', row=1, col=2)
fig.update_xaxes(title_text='Predicciones', row=1, col=3)

fig.show()